<a href="https://colab.research.google.com/github/m-wessler/wr-stid-notebooks/blob/main/other/CoLab_Familiarization_Metadata_Polling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import
Using the ! as an escape character, we can call pip (python package installer) to download and install necessary modules.<br>
Use the "play" button (or Shift+Enter) to run a cell and proceed one cell at a time. Notebooks are generally meant to be run sequentially, in order, as a python script would be.

In [ ]:
!pip install cartopy ipyleaflet ipywidgets branca

In [ ]:
import json
import zipfile
import requests

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.img_tiles import GoogleTiles

from ipyleaflet import Map, Marker, MarkerCluster, TileLayer, basemaps, CircleMarker, LayersControl
from ipywidgets import HTML
import branca.colormap as cm

## User Variables
Set user variables by either directly entering values into the python code, or by using the form to the right (recommended). These forms are used to populate variables across multiple scripts in the STID CoLab toolkit.

In [ ]:
# @markdown Supply your own Synoptic API Token Here
token = '' #@param {type:'string'}
# @markdown Can use a single CWA, a comma-separated list of CWAs (e.g. SLC,PIH,BOI), or single RFC below (e.g. CNRFC)
cwa_rfc = '' #@param {type:'string'}
cwa_rfc = cwa_rfc.upper() # force uppercase
# @markdown Choose which network(s) to display
network = '<select network>' #@param ['<select network>', 'NWS', 'NWS+RAWS', 'NWS+HADS', 'NWS+RAWS+HADS', 'NWS', 'RAWS', 'HADS', 'ALL'] {type:'string'}
# @markdown Select sites with available data for a given variable
variable = '<select variable>' #@param ['<select variable>', 'air_temp' ,'precip', 'humidity', 'wind', 'gust', 'snow', 'cloud'] {type:'string'}

## Script Main
Here we will fetch the data from the Synoptic API, filter it, and move it into a Pandas DataFrame. The following blocks of code will do some basic analysis and produce a set of histograms and a map. Run through each sequentially.

Running blocks of code out of order can cause a number of errors if variables are overwritten. Use "Runtime" > "Restart Session" and "Edit" > "Clear All Outputs" to start over.

In [ ]:
metadata_api = 'https://api.synopticdata.com/v2/stations/metadata?'

# Convert named network to Synoptic API numeric network IDs (see docs)
synoptic_networks = {
    "NWS+RAWS+HADS":"1,2,106",
    "NWS+RAWS":"1,2",
    "NWS+HADS":"1,106",
    "NWS":"1",
    "RAWS": "2",
    "HADS":"106",
    "ALL":None}

network_query = (f"&network={synoptic_networks[network]}"
                 if synoptic_networks[network] is not None else '')

# Assemble the API query
if cwa_rfc == '':
    cwa_str = ''
elif 'RFC' in cwa_rfc:
    cwa_str = ''
else:
    cwa_str = f"&cwa={cwa_rfc}"

api_query = (f"{metadata_api}&token={token}{cwa_str}" + network_query +
            f"&complete=1&sensorvars=1,obrange=20230118")

# Print the API query to output
print(api_query)

# Get the data from the API
response = requests.get(api_query)
metadata = pd.DataFrame(response.json()['STATION'])

# Remove NaNs and index by network, station ID
metadata = metadata[metadata['SHORTNAME'].notna()]
metadata = metadata.set_index(['SHORTNAME', 'STID'])

metadata['LATITUDE'] = metadata['LATITUDE'].astype(float)
metadata['LONGITUDE'] = metadata['LONGITUDE'].astype(float)
metadata['ELEVATION'] = metadata['ELEVATION'].astype(float)

metadata = metadata[metadata['LATITUDE'] >= 31]
metadata = metadata[metadata['LONGITUDE'] <= -103.00]
metadata = metadata[metadata['STATUS'] == 'ACTIVE']

variable_mask = np.array([i for i, md in enumerate(metadata['SENSOR_VARIABLES'])
                                if variable in str(md.keys())])

metadata = metadata.iloc[variable_mask]

geometry = gpd.points_from_xy(metadata.LONGITUDE, metadata.LATITUDE)
metadata = gpd.GeoDataFrame(metadata, geometry=geometry)

if 'RFC' in cwa_rfc:

    req = requests.get(
        'https://www.weather.gov/source/gis/Shapefiles/Misc/rf05mr24.zip',

    allow_redirects=True)
    open('rf05mr24.zip', 'wb').write(req.content)

    with zipfile.ZipFile('rf05mr24.zip', 'r') as zip_ref:
        zip_ref.extractall()

    rfc_shp = gpd.read_file('rf05mr24.shp').set_index('BASIN_ID')

    metadata = metadata[metadata.geometry.within(rfc_shp.geometry.loc['NWRFC'])]

print(metadata.shape)
metadata.head()

In [ ]:
network_names = metadata.index.get_level_values(0).unique()

print('Station Counts:')
for nn in network_names:
    print(nn, metadata.loc[nn, :].shape[0])

In [ ]:
print('Station Elevations:')

fig, axs = plt.subplots(network_names.size, 1, sharex=True,
                        figsize=(8, 1.5*network_names.size))

try:
    axs = axs.flatten()
except:
    axs = [axs]

for i, nn in enumerate(network_names):
    axs[i].hist(metadata.loc[nn, :]['ELEVATION'],
             bins=np.arange(0, np.ceil(metadata.ELEVATION.max()/1000)*1000, 500),
                edgecolor='k', alpha=0.5, width=475,
             label=f'{nn} ({metadata.loc[nn, :].shape[0]})')
    axs[i].grid(zorder=-1)
    axs[i].set_xlim(left=0)
    axs[i].legend()

axs[i].set_xlabel('Elevation (ft)')

fig.subplots_adjust(hspace=0.1)
plt.show()

The cell below leverages geopandas and cartopy to plot the stations across a selected CWA (or multiple) with elevation shaded.

In [ ]:
popup_text_color = "darkblue"
map_width = "900px"
map_height = "600px"

# Define the colormap for elevation
vmin = metadata.ELEVATION.min()
vmax = metadata.ELEVATION.max()
color_map = cm.linear.YlGnBu_09.scale(vmin, vmax)

# Map center
center_lat = metadata.LATITUDE.mean()
center_lon = metadata.LONGITUDE.mean()


# Map setup
m = Map(center=(center_lat, center_lon), zoom=7,
        scroll_wheel_zoom=True,
        layout={'width': map_width, 'height': map_height})

# Tile layers
terrain = TileLayer(url="https://stamen-tiles-{s}.a.ssl.fastly.net/terrain/{z}/{x}/{y}.png",
                   attribution='Map tiles by <a href="http://stamen.com">Stamen Design</a>, ' +
                   'under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>. ' +
                   'Data by <a href="http://openstreetmap.org">OpenStreetMap</a>',
                   name='Stamen Terrain')
osm = TileLayer(url='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
               attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a>',
               name='OpenStreetMap')
satellite = basemaps.Esri.WorldImagery

# OpenTopoMap tile layer
topo = TileLayer(
    url="https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png",
    attribution='Map data: © <a href="https://opentopomap.org">OpenTopoMap</a> contributors',
    name='OpenTopoMap'
)

# Add to map
m.add_layer(topo)
m.add_layer(osm)
# m.add_layer(terrain)
m.add_layer(satellite)
terrain.visible = True
osm.visible = False
satellite.visible = False
m.add_control(LayersControl())

# Add markers
for i, nn in enumerate(network_names):
    network_data = metadata.loc[nn, :]
    for idx, row in network_data.iterrows():
        elevation = row['ELEVATION']
        color = color_map(elevation)
        station_name = row.get('STATION', idx)
        popup_html = f"""
        <div style='width:180px; color:{popup_text_color};'>
            <b>Station:</b> {station_name}<br>
            <b>Network:</b> {nn}<br>
            <b>Elevation:</b> {elevation} ft<br>
            <b>Lat/Lon:</b> {row.geometry.y:.4f}, {row.geometry.x:.4f}
        </div>
        """
        marker = CircleMarker(
            location=(row.geometry.y, row.geometry.x),
            radius=7,
            color="white",
            weight=2,
            fill_color=color,
            fill_opacity=0.8,
            popup=HTML(popup_html)
        )
        m.add_layer(marker)

title_html = f'<h3 style="text-align:center;color:{popup_text_color};">{cwa_rfc.upper()} Stations ({network})</h3>'
title_widget = HTML(title_html)
display(title_widget, m)